# Marginal Cost Extraction from LCOE Excel File
This notebook extracts relevant data from the provided Excel file, computes marginal costs, and interpolates for the years between available data points (e.g., 2020, 2030, 2050). Finally, it saves the extracted and interpolated data into a CSV file.

In [4]:
# Import necessary libraries
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

# Load the Excel file
file_path = '../data/lcoe_calculator_with_ee_module_0.xlsm'

# Load the relevant sheets ('ResultsAllYears')
results_df = pd.read_excel(file_path, sheet_name='ResultsAllYears')

# Filter relevant technologies (thermal)
thermal_keywords = ['coal', 'gas', 'biomass', 'CHP', 'thermal']
thermal_df = results_df[results_df['Technology name'].str.contains('|'.join(thermal_keywords), case=False, na=False)]

# Select relevant columns (Technology name, Year, Fuel cost, O&M costs)
thermal_df = thermal_df[['Technology name', 'Year', 'Fuel cost', 'O&M costs']]

# Calculate Marginal Cost (Fuel cost + O&M costs)
thermal_df['Marginal Cost'] = thermal_df['Fuel cost'] + thermal_df['O&M costs']

thermal_df.head(50)

,Technology name,Year,Fuel cost,O&M costs,Marginal Cost
12,"18 Me BP STR 80MW input, Straw CHP Medium, 2 d...",2015,36.299772,42.481215,78.780987
13,"18 Me BP STR 80MW input, Straw CHP Medium, 2 d...",2020,37.896525,42.401014,80.297539
14,"18 Me BP STR 80MW input, Straw CHP Medium, 2 d...",2030,40.614203,40.010345,80.624548
15,"18 Me BP STR 80MW input, Straw CHP Medium, 2 d...",2050,40.625592,36.032124,76.657716
16,"16 ME BP SC NGA - Gas Turbine Single Cycle, me...",2015,60.521160,9.135643,69.656804
17,"16 ME BP SC NGA - Gas Turbine Single Cycle, me...",2020,63.275650,8.920687,72.196337
18,"16 ME BP SC NGA - Gas Turbine Single Cycle, me...",2030,71.195960,8.512270,79.708230
19,"16 ME BP SC NGA - Gas Turbine Single Cycle, me...",2050,70.049349,8.168340,78.217689
20,16La EX CC NGA CC large CHP Gas Turbine combin...,2015,48.359166,11.448975,59.808140
21,16La EX CC NGA CC large CHP Gas Turbine combin...,2020,51.690425,11.168792,62.859217


In [ ]:

# Pivot the table to get years as columns for easier interpolation
thermal_pivot = thermal_df.pivot_table(index='Technology name', columns='Year', values='Marginal Cost')

# Interpolate for missing years (2020-2050 range)
years = np.arange(2020, 2051)  # Create a range of years from 2020 to 2050
interpolated_df = pd.DataFrame(index=thermal_pivot.index)

# Interpolation function for each technology
for tech in thermal_pivot.index:
    tech_costs = thermal_pivot.loc[tech].dropna()  # Get non-null costs for this technology
    interp_func = interp1d(tech_costs.index, tech_costs.values, kind='linear', fill_value="extrapolate")
    interpolated_df[tech] = interp_func(years)

# Transpose to get years back as rows and technologies as columns
interpolated_df = interpolated_df.T
interpolated_df.columns = thermal_pivot.index

# Save the interpolated marginal costs to a CSV file
interpolated_df.to_csv('interpolated_marginal_costs.csv')

print("Interpolated data has been saved to 'interpolated_marginal_costs.csv'.")
